In [3]:
from parlai.core.agents import create_agent
from parlai.agents.emely.emely import EmelyAgent
from parlai.core.opt import Opt
from pathlib import Path
import torch
from parlai.scripts.torchscript import export_emely
from parlai.core.agents import create_agent

# Initialize model settings

model_path = Path.cwd().parent / 'models/model-runs/major-wood'
assert model_path.is_dir()

opt_path = model_path / 'model.opt'
opt = Opt.load(opt_path)

# Change opts
opt['skip_generation'] = False
opt['init_model'] = (model_path / 'model').as_posix()
opt['no_cuda'] = True  # Cloud run doesn't offer gpu support

# Inference options
opt['inference'] = 'beam' # 'beam' 'greedy'
opt['beam_size'] = 10

opt["scripted_model_file"] = "../saved_models/emely_scripted_test.pt"
opt["script-module"] = "parlai.torchscript.modules:TorchScriptGreedySearch"
opt["model_file"] = opt["init_model"]
opt["temp_separator"] = "__space__"
opt["bpe_add_prefix_space"] = False

In [4]:
# Test exporting to torchscript. This function exports and saves a model at the specified location
quantize = False
original_module, scripted_module = export_emely(opt,quantize)

10:05:56 | loading dictionary from /home/ckjellson/code/emely-models/models/model-runs/major-wood/model.dict
10:05:56 | num words = 54944


/home/ckjellson/anaconda3/envs/emelymodels/lib/python3.8/site-packages/torch/jit/annotations.py:286: UserWarning: TorchScript will treat type annotations of Tensor dtype-specific subtypes as if they are normal Tensors. dtype constraints are not enforced in compilation either.
  warnings.warn("TorchScript will treat type annotations of Tensor "


10:05:56 | DEPRECATED: XLM should only be used for backwards compatibility, as it involves a less-stable layernorm operation.
10:05:57 | Total parameters: 87,508,992 (87,508,992 trainable)
10:05:57 | Loading existing model params from /home/ckjellson/code/emely-models/models/model-runs/major-wood/model


/home/ckjellson/code/emely-models/ParlAI/parlai/agents/emely/emely.py:29: UserWarning: Tried to add persona to agent history but EmelyAgent.persona is None
  warnings.warn('Tried to add persona to agent history but EmelyAgent.persona is None')
/home/ckjellson/code/emely-models/ParlAI/parlai/agents/transformer/modules/encoder.py:183: TracerWarning: Converting a tensor to a Python number might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if positions.max().item() > self.n_positions:
/home/ckjellson/code/emely-models/ParlAI/parlai/agents/transformer/modules/attention.py:147: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inp

FileNotFoundError: [Errno 2] No such file or directory: '../saved_models/emely_scripted_test.pt'

In [4]:
# Create model to test tokenization etc
emely_agent = EmelyAgent(opt)

14:26:45 | loading dictionary from /home/ckjellson/code/emely-models/ParlAI/data/models/blender/blender_90M/model.dict
14:26:45 | num words = 54944
14:26:46 | Total parameters: 87,508,992 (87,508,992 trainable)
14:26:46 | Loading existing model params from /home/ckjellson/code/emely-models/ParlAI/data/models/blender/blender_90M/model


In [9]:
# Make sure that the output is the same
text = "Hi Emely, how are you?\nI'm good thanks! What do you do for work?\nI write code and I drink coffe.\n"

reply = emely_agent.observe_and_act(text)
print(reply)
reply = scripted_module(text)
print(reply)

9797
do you have any hobbies , i ' d like to get to know you a bit .
do you have any hobbies , i like to write code , coffe , how about you ?


In [9]:
text = "Hi Emely, how are you?"
print(emely_agent.dict.txt2vec(text))
print(scripted_module.dict.txt2vec(text))

[792, 813, 3459, 6, 102, 46, 15, 20]
[792, 813, 3459, 6, 102, 46, 15, 20]


In [12]:
# Test performance
from time import time
nruns = 1
t1 = time()
for i in range(nruns):
    reply = emely_agent.observe_and_act("Hi!\n My name is something and my friend's name is something likewise.")
    print(reply)
t2 = time()
orig_time = t2-t1
print("Time using original emely_agent: ", orig_time)
for i in range(nruns):
    reply = original_module("Hi!\n My name is something and my friend's name is something likewise.")
    print(reply)
t3 = time()
orig2_time = t3-t2
print("Time using original module: ", orig2_time)
for i in range(nruns):
    reply = scripted_module("Hi!\n My name is something and my friend's name is something likewise.")
    print(reply)
t4 = time()
script_time = t4-t3
print("Time using scripted model: ", script_time)

that ' s cool . what is your name ? i ' ve never met anyone with the same name .
Time using original emely_agent:  0.8299565315246582
that ' s cool . what is your name ? i ' ve never met anyone with the same name as me .
Time using original module:  0.7110500335693359
that ' s cool . what is your name ? i ' ve never met anyone with the same name as me .
Time using scripted model:  0.7660255432128906


In [6]:
from parlai.core.message import Message
dummy_message = Message()
dummy_message['id'] = 'localHuman'
dummy_message['text'] = 'Hi'
dummy_message['episode_done'] = False
dummy_message['label_candidates'] = None
#scripted_module.observe()

In [8]:
print(type(emely_agent.model.encoder))

<class 'parlai.agents.transformer.modules.encoder.TransformerEncoder'>


In [9]:
print(emely_agent.dict.tokenize("Hi Emely, how are you?"))
print(scripted_module.dict.tokenize("Hi Emely, how are you?"))

['hi', 'em@@', 'ely', ',', 'how', 'are', 'you', '?']
['hi', 'em@@', 'ely', ',', 'how', 'are', 'you', '?']
